# Interactive Cropping Tool Demo

In [1]:
# import interactive_crop
import cropping_widget
import os
image_list = ['image_samples/'+i for i in os.listdir('image_samples')]
cropping_widget.main(image_list)


GridBox(children=(Dropdown(description='Img Name:', layout=Layout(grid_area='im_selector', width='100%'), opti…

In [5]:
print('me')

me
